# Real life data

## Preprocessing

In [1]:
%load_ext autoreload 
%autoreload 2

import re
import sys
import hashlib

import pandas as pd
import numpy as np
from pathlib import Path

# IT'S 2025 WHY HASN'T SOMEONE FIXED THIS
if ".." not in sys.path:
    sys.path.insert(0, "..")
from src import aposteriori

import tasks.run_helper
import tasks.preprocessing

In [2]:
df = pd.read_pickle("../data/attitudes_embedded.csv")
df = df.loc[
    :,
    [
        "tweet",
        "racism",
        "annotatorAge",
        "annotatorRace",
        "annotatorGender",
    ],
]

Clear out NaNs from dataset

In [3]:
nan_age = df.annotatorAge.apply(lambda x: np.any(np.isnan(x)))
nan_race = df.annotatorRace.apply(lambda x: "na" in x)
nan_gender = df.annotatorGender.apply(lambda x: "na" in x)

nan_age.sum(), nan_race.sum(), nan_gender.sum()

(np.int64(41), np.int64(41), np.int64(39))

In [4]:
def process_age_list(x, bins):
    if not isinstance(x, (list, tuple)):
        return None
    if any(pd.isna(age) for age in x):
        return None
    try:
        int_ages = [int(age) for age in x]
        return pd.cut(int_ages, bins=bins, include_lowest=True)
    except Exception:
        return None


all_ages = [
    age
    for sublist in df["annotatorAge"]
    if isinstance(sublist, (list, tuple))
    for age in sublist
    if pd.notna(age)
]
all_ages = list(map(int, all_ages))
num_bins = 4
bin_edges = [0, 20, 40, 60, 80]

df["annotatorAge"] = df["annotatorAge"].apply(
    lambda x: process_age_list(x, bin_edges)
)
df.annotatorRace = df.annotatorRace.apply(lambda x: None if ("na" in x) else x)

df.annotatorGender = df.annotatorGender.apply(
    lambda x: None if ("na" in x) else x
)

df["random"] = df.racism.apply(
    lambda x: [np.random.randint(1, 10) for _ in range(len(x))]
)
df = df.dropna()

In [5]:
tasks.preprocessing.find_inconsistent_rows(
    df, ["annotatorAge", "annotatorRace", "racism", "annotatorGender"]
).sum()

np.int64(0)

## Results

We hypothesize that all of the comments belong in the same discussion.

In [6]:
df2 = df.copy()
df2["fake_index"] = 1

In [7]:
tasks.run_helper.results(
    df2,
    discussion_id_col="fake_index",
    sdb_column="annotatorAge",
    value_col="racism",
    comment_key_col="tweet",
)

1          
                   value    pvalue
(20.0, 40.0]    0.033372  0.654701
(40.0, 60.0]   -0.059469  0.592593
(60.0, 80.0]   -0.788553  1.000000
(-0.001, 20.0]  0.000000  1.000000

In [8]:
tasks.run_helper.results(
    df2,
    discussion_id_col="fake_index",
    sdb_column="annotatorRace",
    value_col="racism",
    comment_key_col="tweet",
)

/home/dimits/conda/envs/apunim-experiments/lib/python3.13/site-packages/numpy/_core/fromnumeric.py:3860: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/dimits/conda/envs/apunim-experiments/lib/python3.13/site-packages/numpy/_core/_methods.py:144: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


1          
                  value    pvalue
black          0.519330  0.260274
white         -0.256693  0.752137
hisp           0.000000  1.000000
other          0.000000  1.000000
middleEastern  0.000000       NaN

In [9]:
tasks.run_helper.results(
    df2,
    discussion_id_col="fake_index",
    sdb_column="annotatorGender",
    value_col="racism",
    comment_key_col="tweet",
)

1          
              value    pvalue
man        0.147123  0.498282
woman      0.047520  0.534050
nonBinary  0.000000  1.000000

In [10]:
tasks.run_helper.results(
    df2,
    discussion_id_col="fake_index",
    sdb_column="random",
    value_col="racism",
    comment_key_col="tweet",
)

1          
      value    pvalue
3 -0.060951  0.766667
8  0.111597  0.759582
1  0.195593  0.750000
4  0.000162  0.726644
9  0.031105  0.784615
6 -0.081510  0.763066
2  0.035875  0.775439
5 -0.207914  0.722807
7  0.126686  0.749117